In [12]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from chatbot import ChatBot, load_api_key
from OktSentiment import Sentiment
from tarot import TarotBot
from refer_text import ResponseSearch

app = Flask(__name__)
CORS(app)

# ChatBot 인스턴스 생성
api_key = load_api_key()
if not api_key:
    raise ValueError("API key not found or invalid. Please check your api_key.txt file.")
respon_search = ResponseSearch('C:\\Users\\SMHRD\\Documents\\카카오톡 받은 파일\\최종_참조자료.csv',"jhgan/ko-sroberta-multitask")
chatbot = ChatBot(api_key, respon_search)  # CSV 파일 경로를 적절히 수정하세요
#C:\Users\SMHRD\Documents\카카오톡 받은 파일\원인-대처종합.csv //c:/Users/SMHRD/원인-대처종합.csv
# Sentiment 인스턴스 생성
sentiment_analyzer = Sentiment()

@app.route("/chatbot", methods=["POST"])
def chat():
    data = request.json
    print("Received data:", data)
    user_nick = data.get("user_nick")
    user_input = data.get("input")
    
    if user_input:
        try:
            response = chatbot.process_input(user_nick, user_input)
            
            if response.get("isProblem"):
                # 형태소 분석 및 감정 분석 수행
                sentiment_results = sentiment_analyzer.parse([user_input])
                # 결과가 딕셔너리 형태로 바꾸고 소수점 소수점 둘쨰자리까지
                if 'polarity' in sentiment_results and len(sentiment_results['polarity']) > 0:
                    sentiment_results = sentiment_results['polarity'][0]
                    sentiment_results = {key: round(value * 100, 2) for key, value in sentiment_results.items()}

                response.update({
                    "sentiment": sentiment_results
                })

            return jsonify(response)
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            return jsonify({"error": f"An error occurred while processing your request: {str(e)}"}), 500
    else:
        return jsonify({"error": "No input provided"}), 400

# 타로카드 라우트
@app.route("/api/interpret",methods=["POST"])

def tarot_card():
    data = request.json
    user_mode = data.get("user_mode")
    cards = data.get("cards")
    user_input = data.get("user_input")
    user_select = data.get("user_select")
    print("recived data: ",data)
    if not cards or len(cards) != 3:
        return jsonify({"error": "카드를 다시 선택해 주세요."}),400 
    
    key_path = "C:\\Users\\SMHRD\\Desktop\\ky_api.txt"
    try:
        tarot = TarotBot(api_key_path=key_path)
        if user_mode =="일반":
            print(user_input)
            response = tarot.general_reading(user_select,cards)
        elif user_mode == "심리":
            response = tarot.interpret_cards(user_input,cards)

        print("response: ",response.get_json())
        return response

    except Exception as e:
        print("Error",str(e))
        return jsonify({"error":str(e)}),500

if __name__ == "__main__":
    app.run(port=7000)

c:\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:7000
Press CTRL+C to quit
127.0.0.1 - - [09/Sep/2024 14:43:56] "OPTIONS /api/interpret HTTP/1.1" 200 -


recived data:  {'user_mode': '일반', 'user_select': '오늘의 운세', 'cards': ['Queen of Pentacles', 'Five of Cups', 'Nine of Pentacles'], 'user_input': None}
None


127.0.0.1 - - [09/Sep/2024 14:44:00] "POST /api/interpret HTTP/1.1" 500 -


Error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Error 'str' object has no attribute 'get_json'
